In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from dataloader import get_dataloaders,get_dataloaders_subword
from utils_subword import load_sentencepiece_model,sp_detokenize_with_specials
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformer import Transformer,TransformerEncoder,TransformerDecoder
import utils
import pickle
nltk.download('punkt')  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/wicaksonolxn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
BATCH_SIZE = 32         #banyakupdate
DATA_PATH = "dataset/"  
DATA_PATH_SPM = "dataset/spm_files"  
SAVE_DIR = "saved"
SRC_VOCAB_SIZE = 5000     
TGT_VOCAB_SIZE = 5000     
N_LAYERS = 1             
N_HEADS = 1
D_MODEL =  128
FFN_HIDDEN = D_MODEL * 4
DROPOUT = 0.3
EPOCHS = 150
sp = load_sentencepiece_model(DATA_PATH_SPM, model_prefix="spm_model")
train_loader, val_loader, test_loader = get_dataloaders_subword(
    sp=sp,
    data_path=DATA_PATH, 
    source_lang="min", 
    target_lang="eng", 
    batch_size=BATCH_SIZE, 
    device=device
)
encoder = TransformerEncoder(SRC_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
decoder = TransformerDecoder(TGT_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
model = Transformer(encoder,decoder,device,utils.PAD_TOKEN).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss(ignore_index=utils.PAD_TOKEN) 
print("Model initialized on:", device)


SentencePiece model loaded from dataset/spm_files/spm_model.model
TrainData - Max 'min' sentence length: 76
TrainData - Max 'eng' sentence length: 107
TestData - Max 'min' sentence length: 61
TestData - Max 'eng' sentence length: 75
ValidData - Max 'min' sentence length: 65
ValidData - Max 'eng' sentence length: 85
Number of examples in train_dataset: 799
Number of examples in valid_dataset: 100
Number of examples in test_dataset: 100
Model initialized on: cuda


In [3]:
tokens = {
    "Padding": utils.PAD_TOKEN,
    "Start of Sequence": utils.SOS_TOKEN,
    "End of Sequence": utils.EOS_TOKEN,
    "Unknown": utils.UNK_TOKEN
}
for i, batch in enumerate(train_loader):
    if i < 8:
        src = batch["src"]
        tgt = batch["tgt"]
        ss,fss=src[0,:],src.shape
        st,fst=tgt[0,:],tgt.shape
        print(fss,fst)
    for name, token in tokens.items():
        print(f"{name}: {token}")
    else:
        break

torch.Size([32, 107]) torch.Size([32, 107])
Padding: 0
Start of Sequence: 1
End of Sequence: 2
Unknown: 3


testing input , is it correct

In [4]:
data_iter = iter(train_loader)
for i in range(1):
    batch = next(data_iter)
    print(batch["tgt"])


tensor([[   1,  910,  622,  ...,    0,    0,    0],
        [   1,  910,   43,  ...,    0,    0,    0],
        [   1,  160,   20,  ...,    0,    0,    0],
        ...,
        [   1,  571, 4977,  ..., 4994, 4977,    0],
        [   1,  195, 4988,  ...,    0,    0,    0],
        [   1,  910,   11,  ...,    0,    0,    0]])


In [5]:
for i, batch in enumerate(train_loader):
    if i < 1:
        src_batch = batch['src'].to(device)  # shape: [batch_size, seq_len]
        tgt_batch = batch['tgt'].to(device)  # shape: [batch_size, seq_len]
        batch_size = src_batch.size(0)
        num_examples = min(5, batch_size)
        for j in range(num_examples):
            src_tokens = src_batch[j].tolist()
            tgt_tokens = tgt_batch[j].tolist()
            src_sentence = sp_detokenize_with_specials(sp, src_tokens)
            tgt_sentence = sp_detokenize_with_specials(sp, tgt_tokens)
            print(f"Example {j+1}:")
            print("Source:", src_sentence)
            print("Target:", tgt_sentence)
            print("-" * 50)
    break


Example 1:
Source: dari segi latak lokasi cukuik mudah dicari  bangunan dan isi bagian di dalamnyo manarik apolai ado taman ketek di dalamnyo nan diisi lo jo gentong yang aianyo dibiaan maalia taruih  mambuek suasananyo jadi tambah romantis dan manyanangan  menu nan disajian lumayan baragam dan dari segi raso lumayan lamak 
Target: the place location is pretty easy to spot  the building and its interior are interesting especially the small garden inside that has an urn with water flowing out of it constantly  makes the mood more romantic and pleasant  the menu offered had a lot of variety
--------------------------------------------------
Example 2:
Source: ko manyasa bana makan indomie salted egg  makannyo tadi pagi jam sampai kini after tastenyo masih ado  ndaklamak bana lo lai ya allah
Target: i really regret eating the salted egg indomie  i ate it this morning at and my mouth can still feel the aftertaste to this very moment  and its really bad dear allah 
-------------------------

In [6]:
for i, batch in enumerate(train_loader):
    if i < 1:
        src_batch = batch['src'].to(device)  # shape: [batch_size, seq_len]
        tgt_batch = batch['tgt'].to(device)  # shape: [batch_size, seq_len]
        batch_size = src_batch.size(0)
        num_examples = min(5, batch_size)
        for j in range(num_examples):
            src_tokens = src_batch[j].tolist()
            tgt_tokens = tgt_batch[j].tolist()
            src_sentence = sp_detokenize_with_specials(sp, src_tokens)
            tgt_sentence = sp_detokenize_with_specials(sp, tgt_tokens)
            print(f"Example {j+1}:")
            print("Source:", src_sentence)
            print("Target:", tgt_sentence)
            print("-" * 50)
    break


Example 1:
Source: gojek ka mulai mambukak layanan nyo di babarapo negara asia tenggara 
Target: gojek will start offering their services in a few sea countries 
--------------------------------------------------
Example 2:
Source: wah lamak bana sate yang lamak bumbu yang pas raso yang pas dan dagiang yang segar sahinggo mahasilan sate yang sabana bana mantap  palayanan yang ramah 
Target: wow the satay is amaing  right spices right seasonings and fresh meat all combined to make a damn good satay  the servers are also friendly 
--------------------------------------------------
Example 3:
Source: indak suko maliek anak sma kini kalau pakai jilbab rambuiknyo pado disangajoan kalua makasuiknyo apo 
Target: dont like seeing highschoolers these days using jilbab but letting the hair out on purpose whats the meaning of that 
--------------------------------------------------
Example 4:
Source: kami mangunjungi tampek ko karano lokasinyo nan badakekan jo tampek kami manginap  kami mamasan n

In [7]:
import os
import torch
from tqdm import tqdm
os.makedirs(SAVE_DIR, exist_ok=True)
best_val_loss = float("inf") 
best_model_path = None
atl = {}
avl = {}
for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")
    model.train()
    total_train_loss = 0.0
    train_bar = tqdm(train_loader, desc="🚀 Training", 
                leave=True, total=len(train_loader))
    for batch in train_bar:
        optimizer.zero_grad()
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        
        output, _ = model(src_batch, tgt_batch[:, :-1]) 
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt_y = tgt_batch[:,1:].contiguous().view(-1)
        loss = criterion(output, tgt_y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        train_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)
    atl[epoch]=avg_train_loss
    model.eval()
    total_val_loss = 0.0
    val_bar = tqdm(val_loader, desc="🚀 Validation",
              leave=True, total=len(val_loader))
    with torch.no_grad():
        for batch in val_bar:
            src_batch = batch['src'].to(device)
            tgt_batch = batch['tgt'].to(device)
            
            output, _ = model(src_batch, tgt_batch[:, :-1]) 
            output_dim = output.shape[-1]
            output = output.reshape(-1, output_dim)
            tgt_y = tgt_batch[:,1:].contiguous().view(-1)

            loss = criterion(output, tgt_y)
            total_val_loss += loss.item()
            val_bar.set_postfix(loss=f"{loss.item():.4f}")
    
    avg_val_loss = total_val_loss / len(val_loader)
    avl[epoch]=avg_val_loss
    print(f"[Epoch {epoch}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    if avg_val_loss < best_val_loss:
        if best_model_path and os.path.exists(best_model_path):
            os.remove(best_model_path)
        best_val_loss = avg_val_loss
        best_model_path = os.path.join(SAVE_DIR, "best_sp.pt")
        torch.save(model.state_dict(), best_model_path)
        print(f"  -> New best model saved at {best_model_path}")

utils.plot_loss(atl, avl, SAVE_DIR,"loss_sp")


Epoch 1/150


🚀 Validation: 100%|██████████| 4/4 [00:00<00:00, 146.39it/s, loss=6.3033]


[Epoch 1] Train Loss: 7.6385 | Val Loss: 6.3228
  -> New best model saved at saved/best_sp.pt
Epoch 2/150


🚀 Validation: 100%|██████████| 4/4 [00:00<00:00, 146.61it/s, loss=5.2933]


[Epoch 2] Train Loss: 5.9029 | Val Loss: 5.3841
  -> New best model saved at saved/best_sp.pt
Epoch 3/150


...

[Epoch 148] Train Loss: 2.9477 | Val Loss: 3.7840
Epoch 149/150


🚀 Validation: 100%|██████████| 4/4 [00:00<00:00, 132.59it/s, loss=3.3915]


[Epoch 149] Train Loss: 2.9398 | Val Loss: 3.7869
Epoch 150/150


🚀 Validation: 100%|██████████| 4/4 [00:00<00:00, 122.05it/s, loss=3.3804]


[Epoch 150] Train Loss: 2.9387 | Val Loss: 3.7830
Loss plot saved to saved/loss_sp.png
